<a href="https://colab.research.google.com/github/himanshu1196/object-tracker/blob/main/obj_trkr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#initialize(kalman_filter)

#for each new img:
  #predicted = predict(kalman_filter)
  #detected = detector(img)
  #visualize predicted and detected
  #if detected:
    #correct(kalman_filter, detected)
  #else:
    #tracked = track(kalman_filter)

In [2]:
# Program To Read video
# and Extract Frames
import cv2
  
# Function to extract frames
def FrameCapture(path, file_name):
      
  # Path to video file
  vidObj = cv2.VideoCapture(path + file_name)

  # Used as counter variable
  count = 0

  # checks whether frames were extracted
  success = 1

  while success:

    # vidObj object calls read
    # function extract frames
    success, image = vidObj.read()
    # Saves the frames with frame-count
    if success:
      cv2.imwrite(f'{path}{file_name[:-4]}/frame{count}.jpg', image)

    count += 1
  print(f'saved {count} frames')

FrameCapture("data/", "ball.mp4")
FrameCapture("data/", "multiObject.avi")

saved 52 frames
saved 42 frames


In [2]:
import tensorflow_hub as hub

# Apply image detector on a batch of image.
detector = hub.load("https://tfhub.dev/tensorflow/efficientdet/lite4/detection/1")


In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.debugging.set_log_device_placement(False)
gpus = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(gpus[0], 'GPU')

Num GPUs Available:  1


In [9]:
from google.colab.patches import cv2_imshow
import numpy as np
import time
import cv2


# image_path = 'data/ball/'
# det_path = 'data/detections/ball/'
# frame_count = 51

image_path = 'data/multiObject/'
det_path = 'data/detections/multiObject/'
frame_count = 41

ball_class_idx = 37
detection_threshold = 0.05
input_dim = (640, 640)
for frame_num in range(frame_count):
  img = cv2.imread(f'{image_path}frame{frame_num}.jpg')
  
  # Display the image 
  # cv2_imshow(img)

  img_resized = cv2.resize(img, input_dim)
  
  model_input = np.expand_dims(img_resized, axis=0)
  
  boxes, scores, classes, num_detections = detector(model_input)
  ball_detections = []
  # print(boxes.shape)
  for i in range(num_detections.numpy()[0]):
    if classes[0][i] == ball_class_idx and scores[0][i] >= detection_threshold:
      ball_detections.append((boxes[0][i].numpy(), scores[0][i].numpy()))
  
  img_with_det = np.copy(img_resized)
  for ball_det in ball_detections:
    bb = ball_det[0]
    # print(bb)
    img_with_det = cv2.rectangle(img_with_det, (int(bb[1]), int(bb[0])), (int(bb[3]), int(bb[2])), (255,10,10), thickness=5)
  cv2.imwrite('%sframe%02d.jpg'%(det_path, frame_num), img_with_det)
  # cv2_imshow(img_with_det)

[202.48425 604.31946 333.7956  638.9688 ]
[200.79109 604.3788  333.3518  638.959  ]
[201.18472 604.13635 333.4241  639.0232 ]
[201.54732 600.80505 335.07794 639.2013 ]
[201.72769 590.26764 337.81256 638.9676 ]
[196.0717  590.0408  340.15408 638.5371 ]
[210.28838 568.6163  335.26617 639.95557]
[180.954        0.34780025 343.89893     54.17791   ]
[212.39496 472.93738 335.97165 563.2246 ]
[1.8551521e+02 2.8143644e-01 3.4443927e+02 5.4738701e+01]
[211.8758  452.63156 336.1012  539.95135]
[1.9445793e+02 1.8850803e-01 3.4025800e+02 9.4726486e+01]
[212.76065 451.28943 336.2726  538.364  ]
[211.9159  429.68582 336.65527 516.9359 ]
[194.24066   13.751626 339.45868  125.78949 ]
[200.62561  52.30748 340.4103  158.98999]
[211.47612 409.2714  336.33936 495.0675 ]
[200.9235    82.861176 339.04587  186.52551 ]
[212.8428  388.54602 336.71454 473.29556]
[200.5784    82.500854 339.72162  186.68332 ]
[213.81813 364.966   336.86578 450.66724]
[198.69202 112.25292 342.34402 215.73509]
[205.8649  142.06134

In [13]:
!pip install ffmpeg-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
import utils

vid_file_name = 'multiObject_det.mp4'
# utils.create_video(frames_patten='data/detections/ball/frame%02d.jpg', video_file = vid_file_name, framerate=8)
utils.create_video(frames_patten='data/detections/multiObject/frame%02d.jpg', video_file = vid_file_name, framerate=8)
utils.show_video(vid_file_name)

In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


cuda:0


In [15]:
#PYTORCH
import torch
import numpy as np
import matplotlib.pyplot as plt

import torchvision.transforms.functional as F
from torchvision.models.detection import fasterrcnn_resnet50_fpn, FasterRCNN_ResNet50_FPN_Weights


plt.rcParams["savefig.bbox"] = 'tight'

def show(imgs):
    if not isinstance(imgs, list):
        imgs = [imgs]
    fig, axs = plt.subplots(ncols=len(imgs), squeeze=False)
    for i, img in enumerate(imgs):
        img = img.detach()
        img = F.to_pil_image(img)
        axs[0, i].imshow(np.asarray(img))
        axs[0, i].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])


model = fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT).to(device)

In [18]:
model.eval()
image_path = 'data/ball/'
det_path = 'data/pytorch_detections/ball/'
ball_class_idx = 34
detection_threshold = 0.05
input_dim = (640, 640)
with torch.no_grad():
  for frame_num in range(41):
    img = cv2.imread(f'{image_path}frame{frame_num}.jpg')
    
    # Display the image 
    # cv2_imshow(img)

    img_resized = cv2.resize(img, input_dim)
    
    #channel first and rescaled to [0,1]
    model_input = torch.tensor(np.expand_dims(np.swapaxes(img_resized, 0, 2)/255.0, axis=0), dtype = torch.float32, requires_grad=False).to(device)

    predictions = model(model_input)
    box_preds = predictions[0]['boxes'].cpu().detach().numpy()
    label_preds = predictions[0]['labels'].cpu().detach().numpy()
    score_preds = predictions[0]['scores'].cpu().detach().numpy()
    num_detections = len(label_preds)
    ball_detections = []

    for i in range(num_detections):
      if label_preds[i] == ball_class_idx and score_preds[i] >= detection_threshold:
        ball_detections.append((box_preds[i], score_preds[i]))
    
    img_with_det = img_resized
    for ball_det in ball_detections:
      bb = ball_det[0]
      img_with_det = cv2.rectangle(img_with_det, (int(bb[1]), int(bb[0])), (int(bb[3]), int(bb[2])), (255,10,10), thickness=5)
    
    cv2.imwrite('%sframe%02d.jpg'%(det_path, frame_num), img_with_det)

time :  0.12102150917053223
time :  0.11102867126464844
time :  0.08395838737487793
time :  0.07931280136108398
time :  0.07635712623596191
time :  0.07657337188720703
time :  0.07223677635192871
time :  0.07621908187866211
time :  0.07496786117553711
time :  0.07520747184753418
time :  0.07771515846252441
time :  0.0786890983581543
time :  0.07768964767456055
time :  0.07851982116699219
time :  0.07758760452270508
time :  0.07829093933105469
time :  0.07970213890075684
time :  0.07914447784423828
time :  0.07774138450622559
time :  0.07944154739379883
time :  0.07807588577270508
time :  0.07942819595336914
time :  0.07701635360717773
time :  0.07762813568115234
time :  0.07829475402832031
time :  0.07757401466369629
time :  0.07740926742553711
time :  0.07799029350280762
time :  0.07749199867248535
time :  0.07988572120666504
time :  0.07918310165405273
time :  0.07747340202331543
time :  0.07834720611572266
time :  0.07819414138793945
time :  0.07685637474060059
time :  0.07740855216

In [19]:
# vid_file_name = 'pytorch_multiObject_det.mp4'
vid_file_name = 'pytorch_ball_det.mp4'
# utils.create_video(frames_patten='data/detections/ball/frame%02d.jpg', video_file = vid_file_name, framerate=8)
utils.create_video(frames_patten='data/pytorch_detections/ball/frame%02d.jpg', video_file = vid_file_name, framerate=8)
utils.show_video(vid_file_name)